In [1]:
import pandas as pd

In [13]:
# basin specific terms "box 3"
basin_terms = pd.read_excel('basins_searchterms_tracking.xlsx') # has the current search terms
# and then country terms new "box 5"
riparian_terms = pd.read_csv('BasinMaster313_20240807.csv') # has riparian names


In [ ]:
basin_code = 'AMZN'

In [ ]:
def create_country_list(basin_code):
    riparian_row = riparian_terms[riparian_terms['BCODE'] == basin_code.upper()]
    riparian_countries_string = riparian_row['Riparian_C'].values[0]
    riparian_countries_list = riparian_countries_string.split(', ')
    return riparian_countries_list

riparian_countries_list = create_country_list(basin_code) # for example
riparian_countries_list

['Bolivia', 'Brazil', 'Colombia', 'Ecuador', 'Guyana', 'Peru', 'Venezuela']

In [10]:
import re

def create_search_string(terms_list):
    # Since terms are already cleaned, just filter out empty/too short terms
    filtered_terms = []
    
    for term in terms_list:
        if isinstance(term, str) and term.strip() and len(term.strip()) > 1:
            filtered_terms.append(term.strip())
    
    # Remove duplicates while preserving order
    unique_terms = []
    for term in filtered_terms:
        if term not in unique_terms:
            unique_terms.append(term)
    
    # Format terms - add quotes only for multi-word terms
    formatted_terms = []
    for term in unique_terms:
        if ' ' in term:
            formatted_terms.append(f'"{term}"')  # Add quotes for multi-word terms
        else:
            formatted_terms.append(term)  # No quotes for single words
    
    # Join with OR
    search_string = " OR ".join(formatted_terms)
    
    return search_string

In [20]:
basin_terms

,BCODE,Basin_Name,Basin_Specific_Terms,Riparian_country_terms,Terms_updated_2025,Downloaded?
0,ADIG,Adige,Adige,NaN,NaN,NaN
1,AKPA,Akpa,Akpa OR Akpakorum OR Ebe OR Ikpan OR Akpasang ...,NaN,Y,NaN
2,ALKL,Alakol,Alakol,NaN,NaN,NaN
3,ALSK,Alsek,Alsek OR Bates Lake,NaN,NaN,NaN
4,AMCR,Amacuro,Amacuro OR Yarakita OR Curimataú OR Tocamabo O...,NaN,Y,NaN
...,...,...,...,...,...,...
309,ZARM,Zarumilla,"Zarumilla OR Chiquita OR ""El Guineo"" OR ""Las L...",NaN,Y,NaN
310,ALHN,Allahein,Allahein OR Allehein,NaN,Y,NaN
311,EMSX,Ems,Ems OR Eems OR Westerwoldse Aa OR Dollart OR D...,NaN,NaN,NaN
312,LOAX,Loa,"Loa OR ""San Pedro de Inacaliri"" OR ""San Salvad...",NaN,Y,NaN


In [17]:
box5_term = create_search_string(riparian_countries_list)
box5_term

'Bolivia OR Brazil OR Colombia OR Ecuador OR Guyana OR Peru OR Venezuela'

In [30]:
def country_terms_to_df(basin_code):
    basin_row = basin_terms[basin_terms['BCODE'] == basin_code.upper()]
    basin_terms.loc[basin_row.index, 'Riparian_country_term'] = box5_term
    basin_terms.to_excel('basins_searchterms_tracking.xlsx') # I knowwwww I'm not supposed to do it like this but I trust it


In [31]:
basins = basin_terms['BCODE'].unique()
for basin in basins:
    riparian_countries_list = create_country_list(basin) # make a list specific to the basin
    box5_term = create_search_string(riparian_countries_list) # turn it into a search string
    country_terms_to_df(basin) # then plop it in the dataframe
    


IndexError: index 0 is out of bounds for axis 0 with size 0

so now the riparian country terms are in the terms list again

In [ ]:
basin_terms = pd.read_excel('basins_searchterms_tracking.xlsx')

In [82]:
# set an example bcode again
basin_code = 'AMZN'

In [ ]:

def get_riparian_search_string(basin_code):

    basin_row = basin_terms[basin_terms['BCODE'] == basin_code.upper()]
    basin_specific_term = basin_row['Basin_Specific_Terms'].values[0]
    riparian_country_terms = basin_row['Riparian_country_term'].values[0]

    box_1_keys = 'water* OR river* OR lake* OR dam* OR stream OR streams OR tributar* OR irrigat* OR flood* OR drought* OR canal* OR hydroelect* OR reservoir* OR groundwater* OR aquifer* OR riparian* OR pond* OR wadi* OR creek* OR oas*s OR spring*'
    box_2_keys = 'treaty OR treaties OR agree* OR negotiat* OR mediat* OR resolv* OR facilitat* OR resolution OR commission* OR council* OR dialog* OR meet* OR discuss* OR secretariat* OR manag* OR peace* OR accord OR settle* OR cooperat* OR collaborat* OR diplomacy OR diplomat* OR statement OR "memo" OR "memos" OR memorand* OR convers* OR convene* OR convention* OR declar* OR allocat*OR share*OR sharing OR apportion* OR distribut* OR ration* OR administ* OR trade* OR trading OR communicat* OR notif* OR trust* OR distrust* OR mistrust*OR support* OR relations* OR consult* OR alliance* OR ally OR allies OR compensat* OR disput* OR conflict* OR disagree* OR sanction* OR war* OR troop* OR skirmish OR hostil* OR attack* OR violen* OR boycott* OR protest* OR clash* OR appeal* OR intent* OR reject* OR threat* OR forc* OR coerc* OR assault* OR fight OR demand* OR disapprov*  OR bomb* OR terror* OR assail* OR insurg* OR counterinsurg* OR destr* OR agitat* OR aggrav* OR veto* OR ban* OR exclud* OR prohibit* OR withdraw* OR suspect* OR combat* OR milit* OR refus* OR deteriorat* OR spurn* OR invad* OR invasion* OR blockad* OR debat* OR refugee* OR migrant* OR violat*'
    box_3_keys = basin_specific_term
    box_4_keys = 'ocean* OR "bilge water" OR "flood of refugees" OR waterproof OR “water resistant” OR streaming OR streame*'
    box_5_keys = riparian_country_terms

    search_string = 'hlead(' + box_1_keys + ') and hlead(' + box_2_keys + ') and hlead(' + box_3_keys + ') and hlead(' + box_5_keys + ') and not hlead(' + box_4_keys + ')'
    # want to check it as-is

    if len(search_string) < 5000: # to see if it fits our search limit
        print("search string is within limit")
        return search_string
    else:
        excess_chars = len(search_string) - 5000 
        remove_from_box3 = len(box_3_keys) - excess_chars
        print(f"string is {len(search_string)} characters, need to create a shorter string")
        box3_truncated = basin_specific_term[:remove_from_box3] # trims it to exactly the limit
        last_or_pos = box3_truncated.rfind(" OR ") # finds the nearest 'or'
        # update the variable
        box_3_keys = basin_specific_term[:last_or_pos] # truncates from there so the final term is a complete one
        search_string = 'hlead(' + box_1_keys + ') and hlead(' + box_2_keys + ') and hlead(' + box_3_keys + ') and hlead(' + box_5_keys + ') and not hlead(' + box_4_keys + ')'
        print(f"new search string is {len(search_string)} characters")
        return search_string

In [ ]:
# write a method here "truncate_basin_specific"
#return basin_specific_term

In [ ]:
get_riparian_search_string(basin_code)

string is 5070 characters, need to create a shorter string
new search string is 4988 characters


'hlead(water* OR river* OR lake* OR dam* OR stream OR streams OR tributar* OR irrigat* OR flood* OR drought* OR canal* OR hydroelect* OR reservoir* OR groundwater* OR aquifer* OR riparian* OR pond* OR wadi* OR creek* OR oas*s OR spring*) and hlead(treaty OR treaties OR agree* OR negotiat* OR mediat* OR resolv* OR facilitat* OR resolution OR commission* OR council* OR dialog* OR meet* OR discuss* OR secretariat* OR manag* OR peace* OR accord OR settle* OR cooperat* OR collaborat* OR diplomacy OR diplomat* OR statement OR "memo" OR "memos" OR memorand* OR convers* OR convene* OR convention* OR declar* OR allocat*OR share*OR sharing OR apportion* OR distribut* OR ration* OR administ* OR trade* OR trading OR communicat* OR notif* OR trust* OR distrust* OR mistrust*OR support* OR relations* OR consult* OR alliance* OR ally OR allies OR compensat* OR disput* OR conflict* OR disagree* OR sanction* OR war* OR troop* OR skirmish OR hostil* OR attack* OR violen* OR boycott* OR protest* OR clash*